# 1. Criação do catálogo
SQL para criação do catálogo e adição de uma descrição.

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS conecta
COMMENT 'Datasets de alguns processos da Concessionária de Iluminação Pública Conecta.
Criado para auxiliar na tomada de decições e utilização de ferramentas OLAP, através de implementação de DWs e possivelmente Data Lakes.'

# 2 Criação do esquema
Inicialmente será criado um esquema chamado alarmes_aju. Esse esquema será modelado para se comportar como um Data Warehouse, no modelo multidimensional tipo estrela, com uma tabela Fato e quatro tabelas dimensionais.

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS conecta.alarmes_aju
COMMENT 'Schema modelado como um Data Warehouse tipo estrela. Tabelas: FatoAlarme, DimData, DimPontoServico, DimTipoAlarme, DimLCU.
Será utilizado para armazenar de forma estrutura em um DW informações de alarmes do sistema de telegestão de Aracaju.\n
As colunas StartDate, EndData e IsLateArriving são usados para controle de atualização de dados em todas tabelas dimensão.'

# 3. Criação das delta tables
Delta tables para representar um Data Warehouse seguindo a modelagem tipo estrela.
![DiagramaEstrela v2.png](./DiagramaEstrela v2.png "DiagramaEstrela v2.png")


In [0]:
%sql
CREATE TABLE IF NOT EXISTS conecta.alarmes_aju.DimTipoAlarme(
  AlarmeKey BIGINT GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
  NomeAlarme STRING,
  StartDate TIMESTAMP NOT NULL,
  EndDate TIMESTAMP,
  IsLateArriving INT NOT NULL
)
COMMENT 'Tabela dimensional DimTipoAlarme. Nessa tabela será armazenado os possíveis tipos de alarmes que podem ser encontrados no sistema de telegestão de Aracaju.
\nNomeAlarme: Indica o nome do alarme so sistema de telegestão.
\nStartDate: Data de criação do registro.
\nEndDate: Data final de um registro.
\nIsLateArriving: Indica se o registro entrou na base de dados durante o processo de ETL da tabela Fato.';
ALTER TABLE conecta.alarmes_aju.DimTipoAlarme
SET TBLPROPERTIES('delta.feature.allowColumnDefaults' = 'supported');
ALTER TABLE conecta.alarmes_aju.DimTipoAlarme
ALTER COLUMN StartDate SET DEFAULT current_timestamp();
ALTER TABLE conecta.alarmes_aju.DimTipoAlarme
ALTER COLUMN IsLateArriving SET DEFAULT 0;

In [0]:
%sql
ALTER TABLE conecta.alarmes_aju.DimTipoAlarme
ALTER COLUMN AlarmeKey COMMENT 'Surrogate';

ALTER TABLE conecta.alarmes_aju.DimTipoAlarme
ALTER COLUMN NomeAlarme COMMENT 'Indica o nome do alarme so sistema de telegestão.';

ALTER TABLE conecta.alarmes_aju.DimTipoAlarme
ALTER COLUMN StartDate COMMENT 'Timestamp da criação do registro.';

ALTER TABLE conecta.alarmes_aju.DimTipoAlarme
ALTER COLUMN EndDate COMMENT 'Timestamp de versionamento do registro.';

ALTER TABLE conecta.alarmes_aju.DimTipoAlarme
ALTER COLUMN IsLateArriving COMMENT 'Marca se o registro entrou na base de dados durante o processo de ETL da tabela Fato. Essa coluna foi criado por recomendação do documento do Databricks, porém não está sendo utilizada na implementação do DW.';

In [0]:
%sql
CREATE TABLE IF NOT EXISTS conecta.alarmes_aju.DimPontoServico(
  PontoServicoKey BIGINT GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
  IDPontoServico INT,
  Bairro STRING,
  Latitude DOUBLE,
  Longitude DOUBLE,
  MarcoContrato STRING,
  AlturaInstalacaoLuminaria DOUBLE,
  StartDate TIMESTAMP NOT NULL,
  EndDate TIMESTAMP,
  IsLateArriving INT NOT NULL,
  TipoPoste STRING
)
COMMENT 'Tabela dimensional DimPontoServico. Nessa tabela será armazenado os dados de pontos de serviço. Os pontos de serviço são postes de iluminação pública de todo tipo, em que são fixados uma ou mais luminárias de iluminação pública. Isso significa que a dimensão de LCU pode ter mais de um registro apontando para um mesmo registro de ponto de serviço.
\nIDPontoServico: Identificador natural único de um ponto de serviço.
\nBairro: Bairro onde o ponto de serviço se encontra.
\nLatitude: Latitude do ponto de serviço.
\nLongitude: Longitude do ponto de serviço.
\nMarcoContrato: Marco do contrato - I ou II e III - em que o ponto de serviço está associado.
\AlturaInstalacaoLuminaria: Altura de instalação média da luminária de iluminação pública.
\nStartDate: Data de criação do registro.
\nEndDate: Data final de um registro.
\nIsLateArriving: Indica se o registro entrou na base de dados durante o processo de ETL da tabela Fato.';

ALTER TABLE conecta.alarmes_aju.DimPontoServico
SET TBLPROPERTIES('delta.feature.allowColumnDefaults' = 'supported');
ALTER TABLE conecta.alarmes_aju.DimPontoServico
ALTER COLUMN StartDate SET DEFAULT current_timestamp();
ALTER TABLE conecta.alarmes_aju.DimPontoServico
ALTER COLUMN IsLateArriving SET DEFAULT 0;

In [0]:
%sql
ALTER TABLE conecta.alarmes_aju.dimpontoservico
ALTER COLUMN PontoServicoKey COMMENT 'Surrogate';

ALTER TABLE conecta.alarmes_aju.dimpontoservico
ALTER COLUMN IDPontoServico COMMENT 'Identificador natural único de um ponto de serviço.';

ALTER TABLE conecta.alarmes_aju.dimpontoservico
ALTER COLUMN Bairro COMMENT 'Bairro onde o ponto de serviço se encontra.';

ALTER TABLE conecta.alarmes_aju.dimpontoservico
ALTER COLUMN Latitude COMMENT 'Latitude do ponto de serviço.';

ALTER TABLE conecta.alarmes_aju.dimpontoservico
ALTER COLUMN Longitude COMMENT 'Longitude do ponto de serviço.';

ALTER TABLE conecta.alarmes_aju.dimpontoservico
ALTER COLUMN MarcoContrato COMMENT 'Marco do contrato - I ou II e III - em que o ponto de serviço está associado.';

ALTER TABLE conecta.alarmes_aju.dimpontoservico
ALTER COLUMN AlturaInstalacaoLuminaria COMMENT 'Altura de instalação média da luminária de iluminação pública.';

ALTER TABLE conecta.alarmes_aju.dimpontoservico
ALTER COLUMN TipoPoste COMMENT 'Indica se o o poste é baixo ou alto. Necessário para a operação, pois define qual equipamento é necessário para dar manutenção no ponto de serviço.';

ALTER TABLE conecta.alarmes_aju.dimpontoservico
ALTER COLUMN StartDate COMMENT 'Timestamp da criação do registro.';

ALTER TABLE conecta.alarmes_aju.dimpontoservico
ALTER COLUMN EndDate COMMENT 'Timestamp de versionamento do registro.';

ALTER TABLE conecta.alarmes_aju.dimpontoservico
ALTER COLUMN IsLateArriving COMMENT 'Marca se o registro entrou na base de dados durante o processo de ETL da tabela Fato. Essa coluna foi criado por recomendação do documento do Databricks, porém não está sendo utilizada na implementação do DW.';

In [0]:
%sql
CREATE TABLE IF NOT EXISTS conecta.alarmes_aju.DimData(
  DataKey BIGINT GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
  Data DATE,
  Mes STRING,
  Ano INT,
  Trimestre STRING,
  TrimestreContrato STRING,
  StartDate TIMESTAMP NOT NULL,
  EndDate TIMESTAMP,
  IsLateArriving INT NOT NULL
)
COMMENT 'Tabela dimensional DimData. Nessa tabela será armazenado os dados de datas.
\nData: Data do alarme.
\nMes: Mês do alarme.
\nAno: Ano do alarme.
\nTrimestre: Trimestre do alarme.
\nTrimestreContrato: Trimestre do contrato. Os trimestres do contrato são enumerados da seguinte forma: T01, T02, T03... Tn. O início do primeiro trimestre do contrato foi em dezembro de 2021, o que significa que a mensuração dos trimestres do contrato seguem uma lógica diferente ao do trimestre normal.
\nStartDate: Data de criação do registro.
\nEndDate: Data final de um registro.
\nIsLateArriving: Indica se o registro entrou na base de dados durante o processo de ETL da tabela Fato.';

ALTER TABLE conecta.alarmes_aju.DimData
SET TBLPROPERTIES('delta.feature.allowColumnDefaults' = 'supported');
ALTER TABLE conecta.alarmes_aju.DimData
ALTER COLUMN StartDate SET DEFAULT current_timestamp();
ALTER TABLE conecta.alarmes_aju.DimData
ALTER COLUMN IsLateArriving SET DEFAULT 0;

In [0]:
%sql
ALTER TABLE conecta.alarmes_aju.DimData
ALTER COLUMN DataKey COMMENT 'Surrogate';

ALTER TABLE conecta.alarmes_aju.DimData
ALTER COLUMN Data COMMENT 'Data do alarme.';

ALTER TABLE conecta.alarmes_aju.DimData
ALTER COLUMN Mes COMMENT 'Mês do alarme com a informação do ano.';

ALTER TABLE conecta.alarmes_aju.DimData
ALTER COLUMN Ano COMMENT 'Ano do alarme.';

ALTER TABLE conecta.alarmes_aju.DimData
ALTER COLUMN Trimestre COMMENT 'Trimestre do alarme.';

ALTER TABLE conecta.alarmes_aju.DimData
ALTER COLUMN TrimestreContrato COMMENT 'Trimestre do contrato. Os trimestres do contrato são enumerados da seguinte forma: T01, T02, T03... Tn. O início do primeiro trimestre do contrato foi em dezembro de 2021, o que significa que a mensuração dos trimestres do contrato seguem uma lógica diferente ao do trimestre normal.';

ALTER TABLE conecta.alarmes_aju.DimData
ALTER COLUMN StartDate COMMENT 'Timestamp da criação do registro.';

ALTER TABLE conecta.alarmes_aju.DimData
ALTER COLUMN EndDate COMMENT 'Timestamp de versionamento do registro.';

ALTER TABLE conecta.alarmes_aju.DimData
ALTER COLUMN IsLateArriving COMMENT 'Marca se o registro entrou na base de dados durante o processo de ETL da tabela Fato. Essa coluna foi criado por recomendação do documento do Databricks, porém não está sendo utilizada na implementação do DW.';


In [0]:
%sql
CREATE TABLE IF NOT EXISTS conecta.alarmes_aju.DimLCU(
  LCUKey BIGINT GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
  IDLCU BIGINT,
  CodigoBarras STRING,
  NomeLCU STRING,
  Latitude DOUBLE,
  Longitude DOUBLE,
  DistanciaPontoServico DOUBLE,
  StartDate TIMESTAMP NOT NULL,
  EndDate TIMESTAMP,
  IsLateArriving INT NOT NULL
)
COMMENT 'Tabela dimensional DimLCU. Nessa tabela será armazenado os dados de LCU, que são os dispositivos IoT instalados em luminárias de iluminação pública. Mais de um registro dessa tabela dimensional pode apontar para um mesmo registro de ponto de serviço.
\nIDLCU: Identificador natural único de uma LCU - dispositivo IoT.
\nCodigoBarras: Código de barras da LCU.
\nNomeLCU: Nome da LCU.
\nLatitude: Latitude da LCU.
\nLongitude: Longitude da LCU.
\nStartDate: Data de criação do registro.
\nEndDate: Data final de um registro.
\nIsLateArriving: Indica se o registro entrou na base de dados durante o processo de ETL da tabela Fato.';

ALTER TABLE conecta.alarmes_aju.DimLCU
SET TBLPROPERTIES('delta.feature.allowColumnDefaults' = 'supported');
ALTER TABLE conecta.alarmes_aju.DimLCU
ALTER COLUMN StartDate SET DEFAULT current_timestamp();
ALTER TABLE conecta.alarmes_aju.DimLCU
ALTER COLUMN IsLateArriving SET DEFAULT 0;

In [0]:
%sql
ALTER TABLE conecta.alarmes_aju.DimLCU
ALTER COLUMN LCUKey COMMENT 'Surrogate';

ALTER TABLE conecta.alarmes_aju.DimLCU
ALTER COLUMN IDLCU COMMENT 'Identificador natural único de uma LCU - dispositivo IoT.';

ALTER TABLE conecta.alarmes_aju.DimLCU
ALTER COLUMN CodigoBarras COMMENT 'Código de barras da LCU.';

ALTER TABLE conecta.alarmes_aju.DimLCU
ALTER COLUMN NomeLCU COMMENT 'Nome da LCU.';

ALTER TABLE conecta.alarmes_aju.DimLCU
ALTER COLUMN Latitude COMMENT 'Latitude da LCU.';

ALTER TABLE conecta.alarmes_aju.DimLCU
ALTER COLUMN Longitude COMMENT 'Longitude da LCU.';

ALTER TABLE conecta.alarmes_aju.DimLCU
ALTER COLUMN DistanciaPontoServico COMMENT 'A distância entre as coordenadas da LCU para as coodernadas do ponto de serviço ao qual a LCU foi associada.';

ALTER TABLE conecta.alarmes_aju.DimLCU
ALTER COLUMN StartDate COMMENT 'Timestamp da criação do registro.';

ALTER TABLE conecta.alarmes_aju.DimLCU
ALTER COLUMN EndDate COMMENT 'Timestamp de versionamento do registro.';

ALTER TABLE conecta.alarmes_aju.DimLCU
ALTER COLUMN IsLateArriving COMMENT 'Marca se o registro entrou na base de dados durante o processo de ETL da tabela Fato. Essa coluna foi criado por recomendação do documento do Databricks, porém não está sendo utilizada na implementação do DW.';

In [0]:
%sql
CREATE TABLE IF NOT EXISTS conecta.alarmes_aju.FatoAlarme(
  AlarmeKey BIGINT NOT NULL,
  DataKey BIGINT NOT NULL,
  LCUKey BIGINT NOT NULL,
  PontoServicoKey BIGINT NOT NULL,
  DateTimeUltimaModificacao TIMESTAMP NOT NULL
)
COMMENT 'Tabela FatoAlarme. Nessa tabela serão registrados os alarmes provenientes do sistema de telegestão de Aracaju. Cada registro representa um alarme, em determinada LCU, em um ponto de serviço em uma data específica, com granularidade diária.
\nEssa tabela Fato não possui dados númericos ou aditivos. Os Fatos analisádos para esse tipo de processo são originados a partir da contagem de alarmes em diferentes tipos de consultas possíveis.';

ALTER TABLE conecta.alarmes_aju.FatoAlarme
SET TBLPROPERTIES('delta.feature.allowColumnDefaults' = 'supported');
ALTER TABLE conecta.alarmes_aju.FatoAlarme
ALTER COLUMN DateTimeUltimaModificacao SET DEFAULT current_timestamp();

In [0]:
%sql
ALTER TABLE conecta.alarmes_aju.fatoalarme
ADD CONSTRAINT FK_FatoAlarme_DimData FOREIGN KEY (DataKey) REFERENCES conecta.alarmes_aju.DimData (DataKey);
ALTER TABLE conecta.alarmes_aju.fatoalarme
ADD CONSTRAINT FK_FatoAlarme_DimLCU FOREIGN KEY (LCUKey) REFERENCES conecta.alarmes_aju.DimLCU (LCUKey);
ALTER TABLE conecta.alarmes_aju.fatoalarme
ADD CONSTRAINT FK_FatoAlarme_DimPontoServico FOREIGN KEY (PontoServicoKey) REFERENCES conecta.alarmes_aju.DimPontoServico (PontoServicoKey);
ALTER TABLE conecta.alarmes_aju.fatoalarme
ADD CONSTRAINT FK_FatoAlarme_DimTipoAlarme FOREIGN KEY (AlarmeKey) REFERENCES conecta.alarmes_aju.DimTipoAlarme (AlarmeKey);

In [0]:
%sql
ALTER TABLE conecta.alarmes_aju.fatoalarme
ALTER COLUMN DataKey COMMENT 'Surrogate da tabela dimensão DimData.';

ALTER TABLE conecta.alarmes_aju.fatoalarme
ALTER COLUMN LCUKey COMMENT 'Surrogate da tabela dimensão DimLCU.';

ALTER TABLE conecta.alarmes_aju.fatoalarme 
ALTER COLUMN PontoServicoKey COMMENT 'Surrogate da tabela dimensão DimPontoServico.';

ALTER TABLE conecta.alarmes_aju.fatoalarme
ALTER COLUMN AlarmeKey COMMENT 'Surrogate da tabela dimensão DimTipoAlarme.';

ALTER TABLE conecta.alarmes_aju.fatoalarme
ALTER COLUMN DateTimeUltimaModificacao COMMENT 'Coluna de timestamp fundamental para o processo de ETL. A informação de timestamp mais recente é utilizada para buscar de fontes de dados operacionais dados recentes, otimizando o processo.';